<a href="https://colab.research.google.com/github/OmAvhad/legal-ai/blob/main/Question_Answering_System_Using_T5small.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import torch
print("PyTorch CUDA available:", torch.cuda.is_available())
print("Number of GPUs:", torch.cuda.device_count())
if torch.cuda.is_available():
    print("GPU Name:", torch.cuda.get_device_name(0))


PyTorch CUDA available: True
Number of GPUs: 1
GPU Name: Tesla T4


In [1]:
!nvidia-smi

Wed Mar  5 05:53:48 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   69C    P8             12W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [5]:
!pip install torch transformers datasets scikit-learn pandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 117.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 84.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 53.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 90.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 40.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [19]:
import pandas as pd

def load_dataset(file_path):
    """
    Load dataset from an Excel file with 'question' and 'answer' columns.

    Parameters:
    file_path (str): Path to the Excel file

    Returns:
    pd.DataFrame: DataFrame with 'source_Text' and 'target_Text' columns
    """
    try:
        # Read the Excel file
        df = pd.read_excel(file_path)

        # Rename columns to match the required format
        df = df.rename(columns={
            'question': 'source_Text',
            'answer': 'target_Text'
        })

        # Validate the columns
        required_columns = ['source_Text', 'target_Text']
        if not all(col in df.columns for col in required_columns):
            raise ValueError(f"Excel file must contain columns: {required_columns}")

        return df

    except Exception as e:
        print(f"Error loading dataset: {e}")
        return None

# Example usage
file_path = 'ipc_qa.xlsx'  # Update with the correct path if needed
df = load_dataset(file_path)

if df is not None:
    print(df.head())  # Display first few rows
    print("\nDataset information:")
    print(df.info())

                                         source_Text  \
0  What is the title and extent of operation of t...   
1  What is the title and extent of operation of t...   
2  What is the title and extent of operation of t...   
3  Where does the operation of 'The Indian Penal ...   
4  Where does the operation of 'The Indian Penal ...   

                                         target_Text  
0  The title is 'The Indian Penal Code' and its o...  
1  The title is 'The Indian Penal Code' and its o...  
2  The title is 'The Indian Penal Code' and its o...  
3  The operation of 'The Indian Penal Code' exten...  
4  The operation of 'The Indian Penal Code' exten...  

Dataset information:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6801 entries, 0 to 6800
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   source_Text  6801 non-null   object
 1   target_Text  6801 non-null   object
dtypes: object(2)
memory usage: 106.4+ KB

In [20]:
print(df)

                                            source_Text  \
0     What is the title and extent of operation of t...   
1     What is the title and extent of operation of t...   
2     What is the title and extent of operation of t...   
3     Where does the operation of 'The Indian Penal ...   
4     Where does the operation of 'The Indian Penal ...   
...                                                 ...   
6796  When did the substitution for certain words co...   
6797  When did the substitution for certain words co...   
6798                         What happened on 1-1-1956?   
6799                         What happened on 1-1-1956?   
6800                         What happened on 1-1-1956?   

                                            target_Text  
0     The title is 'The Indian Penal Code' and its o...  
1     The title is 'The Indian Penal Code' and its o...  
2     The title is 'The Indian Penal Code' and its o...  
3     The operation of 'The Indian Penal Code' exten...  
4

In [26]:
import torch
from datasets import load_dataset, Dataset
from transformers import T5Tokenizer, T5ForConditionalGeneration, Trainer, TrainingArguments
import pandas as pd
from sklearn.model_selection import train_test_split
import subprocess

# Diagnostic GPU checks
def check_gpu_availability():
    print("PyTorch CUDA available:", torch.cuda.is_available())

    try:
        # List CUDA devices
        print("\nCUDA Devices:")
        print(torch.cuda.device_count(), "CUDA device(s) available")

        # Print GPU information
        for i in range(torch.cuda.device_count()):
            print(f"Device {i}: {torch.cuda.get_device_name(i)}")

        # Check system-level GPU information
        print("\nNVIDIA-SMI Output:")
        nvidia_smi_output = subprocess.check_output(["nvidia-smi"]).decode('utf-8')
        print(nvidia_smi_output)
    except Exception as e:
        print("Error getting GPU information:", e)

# Run GPU availability check
check_gpu_availability()

# Load and prepare dataset
# def load_squad_dataset():
#     squad = load_dataset('squad', split='train')

#     data = []
#     for item in squad:
#         source_text = f"question: {item['question']} context: {item['context']}"
#         target_text = item['answers']['text'][0]

#         data.append({
#             'source_text': source_text,
#             'target_text': target_text
#         })

#     return pd.DataFrame(data)

# # Load dataset
# df = load_squad_dataset()

# Split the dataset
train_df, test_df = train_test_split(df, test_size=0.2)

# Determine device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"\nUsing device: {device}")

# Load tokenizer and model
model_name = "t5-small"
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)

# Explicitly move model to device
model = model.to(device)

# Tokenization function
def tokenize_function(examples):
    # Tokenize inputs
    inputs = tokenizer(
        examples['source_Text'],
        max_length=512,
        truncation=True,
        padding="max_length",
        return_tensors="pt"
    )

    # Tokenize targets
    targets = tokenizer(
        examples['target_Text'],
        max_length=128,
        truncation=True,
        padding="max_length",
        return_tensors="pt"
    )

    # Move tensors to device
    inputs = {k: v.to(device) for k, v in inputs.items()}
    targets = {k: v.to(device) for k, v in targets.items()}

    inputs["labels"] = targets["input_ids"]
    return inputs

# Convert to Hugging Face Dataset
train_dataset = Dataset.from_pandas(train_df[:5000])
eval_dataset = Dataset.from_pandas(test_df[:1000])

print(train_dataset)

# Tokenize datasets
train_dataset = train_dataset.map(tokenize_function, batched=True)
eval_dataset = eval_dataset.map(tokenize_function, batched=True)

# Set up training arguments with more GPU-specific configurations
training_args = TrainingArguments(
    output_dir="./qa_results",
    num_train_epochs=1,  # Reduced for faster testing
    per_device_train_batch_size=4,  # Reduced batch size
    per_device_eval_batch_size=4,
    warmup_steps=100,
    weight_decay=0.01,
    logging_dir="./qa_logs",
    logging_steps=10,
    evaluation_strategy="epoch",
    report_to="none",
    fp16=torch.cuda.is_available(),
    dataloader_num_workers=2,
)

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset
)

# Train the model
trainer.train()

# Save the model
trainer.save_model("./t5_small_qa")

PyTorch CUDA available: True

CUDA Devices:
1 CUDA device(s) available
Device 0: Tesla T4

NVIDIA-SMI Output:
Wed Mar  5 14:44:53 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   67C    P0             30W /   70W |     812MiB /  15360MiB |      0%      Default |
|                             

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Epoch,Training Loss,Validation Loss
1,0.591200,0.497909


In [6]:
pip install torch transformers pandas

In [27]:
import torch
from transformers import T5Tokenizer, T5ForConditionalGeneration

def save_model_and_tokenizer(model, tokenizer, save_path):
    """
    Explicitly save both the model and tokenizer

    Args:
        model (T5ForConditionalGeneration): Trained model
        tokenizer (T5Tokenizer): Tokenizer
        save_path (str): Directory to save model and tokenizer
    """
    # Ensure the save directory exists
    import os
    os.makedirs(save_path, exist_ok=True)

    # Save the model
    model.save_pretrained(save_path)

    # Save the tokenizer
    tokenizer.save_pretrained(save_path)

    print(f"Model and tokenizer saved to {save_path}")

def load_model_and_tokenizer(model_path):
    """
    Load the saved model and tokenizer

    Args:
        model_path (str): Path to the saved model

    Returns:
        tuple: Loaded model and tokenizer
    """
    # First, try loading from the specific path
    try:
        tokenizer = T5Tokenizer.from_pretrained(model_path)
        model = T5ForConditionalGeneration.from_pretrained(model_path)
    except:
        # If that fails, fall back to the original model
        print("Falling back to original model loading")
        tokenizer = T5Tokenizer.from_pretrained("t5-small")
        model = T5ForConditionalGeneration.from_pretrained("t5-small")

    # Move model to GPU if available
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device)
    model.eval()

    return model, tokenizer

# Modify your original training script to include this:
# After training, add these lines:
save_path = "/content/t5_small_qa"
save_model_and_tokenizer(model, tokenizer, save_path)

Model and tokenizer saved to /content/t5_small_qa


In [28]:
import torch
from transformers import T5Tokenizer, T5ForConditionalGeneration

def answer_question(model, tokenizer, question, context, max_length=100):
    """
    Generate an answer for a given question and context

    Args:
        model (T5ForConditionalGeneration): Trained model
        tokenizer (T5Tokenizer): Tokenizer
        question (str): Input question
        context (str): Context for the question
        max_length (int): Maximum length of generated answer

    Returns:
        str: Generated answer
    """
    # Prepare input text
    input_text = f"question: {question} context: {context}"

    # Tokenize input
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    inputs = tokenizer(input_text, return_tensors="pt", max_length=512, truncation=True).to(device)

    # Generate answer
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_length=max_length,
            num_return_sequences=1,
            num_beams=4,
            early_stopping=True
        )

    # Decode the generated answer
    answer = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return answer

def main():
    # Load the saved model
    model_path = "./t5_small_qa"  # Adjust path as needed
    tokenizer = T5Tokenizer.from_pretrained(model_path)
    model = T5ForConditionalGeneration.from_pretrained(model_path)

    # Move model to GPU if available
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device)
    model.eval()

    # Test cases specific to Indian Penal Code
    test_cases = [
        {
            'question': 'what is section 74',
            'context': ''
        },
        {
            'question': 'What is the extent of operation of the Indian Penal Code?',
            'context': 'The Indian Penal Code extends to the punishment of offences committed within India, and beyond but which by law may be tried within India. It also includes extension of the Code to extra-territorial offences.'
        },
        {
            'question': 'What does the Indian Penal Code say about laws not to be affected by this Act?',
            'context': 'The Indian Penal Code specifies that certain laws are not to be affected by this Act.'
        },
        {
            'question': 'What does section 68 of the Indian Penal Code state about imprisonment?',
            'context': 'Section 68 states that imprisonment is to terminate on payment of fine.'
        },
        {
            'question': 'How does the Indian Penal Code handle offences committed outside India?',
            'context': 'The Indian Penal Code extends to the punishment of offences committed within India, and beyond but which by law may be tried within India. It also includes extension of the Code to extra-territorial offences.'
        }
    ]

    # Test the model
    print("Question Answering Results for Indian Penal Code:")
    for case in test_cases:
        print("\nQuestion:", case['question'])
        print("Context:", case['context'])
        answer = answer_question(model, tokenizer, case['question'], case['context'])
        print("Generated Answer:", answer)

if __name__ == "__main__":
    main()

Question Answering Results for Indian Penal Code:

Question: what is section 74
Context: 
Generated Answer: 74

Question: What is the extent of operation of the Indian Penal Code?
Context: The Indian Penal Code extends to the punishment of offences committed within India, and beyond but which by law may be tried within India. It also includes extension of the Code to extra-territorial offences.
Generated Answer: extension of the Code to extra-territorial offences

Question: What does the Indian Penal Code say about laws not to be affected by this Act?
Context: The Indian Penal Code specifies that certain laws are not to be affected by this Act.
Generated Answer: certain laws are not to be affected by this Act.

Question: What does section 68 of the Indian Penal Code state about imprisonment?
Context: Section 68 states that imprisonment is to terminate on payment of fine.
Generated Answer: imprisonment is to terminate on payment of fine

Question: How does the Indian Penal Code handle o

In [2]:
!pip install rouge_score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=cf9f5cfa76b02de97ab74e3572c480525c25942156331c4fe67d434e453c9f72
  Stored in directory: /root/.cache/pip/wheels/1e/19/43/8a442dc83660ca25e163e1bd1f89919284ab0d0c1475475148
Successfully built rouge_score


In [34]:
!pip install fuzzywuzzy

In [35]:
import torch
import numpy as np
from transformers import T5Tokenizer, T5ForConditionalGeneration
from rouge_score import rouge_scorer
from fuzzywuzzy import fuzz

def calculate_accuracy(true_answers, predicted_answers, threshold=70):
    """
    Calculate accuracy using multiple metrics

    Args:
        true_answers (list): List of ground truth answers
        predicted_answers (list): List of model-generated answers
        threshold (int): Fuzzy matching threshold (0-100)

    Returns:
        dict: Accuracy metrics
    """
    # Exact match accuracy
    exact_matches = sum(t.lower().strip() == p.lower().strip() for t, p in zip(true_answers, predicted_answers))
    exact_match_accuracy = exact_matches / len(true_answers) * 100

    # Fuzzy match accuracy
    fuzzy_matches = sum(
        fuzz.ratio(t.lower().strip(), p.lower().strip()) >= threshold
        for t, p in zip(true_answers, predicted_answers)
    )
    fuzzy_match_accuracy = fuzzy_matches / len(true_answers) * 100

    # ROUGE-L accuracy (using F1 score)
    rouge_scorer_instance = rouge_scorer.RougeScorer(['rougeL'], use_stemmer=True)
    rouge_scores = [
        rouge_scorer_instance.score(t, p)['rougeL'].fmeasure
        for t, p in zip(true_answers, predicted_answers)
    ]
    average_rouge_l = np.mean(rouge_scores) * 100

    return {
        'exact_match_accuracy': exact_match_accuracy,
        'fuzzy_match_accuracy': fuzzy_match_accuracy,
        'average_rouge_l': average_rouge_l
    }

def evaluate_model(model, tokenizer, test_cases, max_length=100):
    """
    Evaluate model performance with manually provided context
    """
    # Prepare device
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device)
    model.eval()

    # Prepare lists to store results
    predicted_answers = []
    true_answers = []

    # Generate predictions
    with torch.no_grad():
        for case in test_cases:
            # Prepare input with context
            input_text = f"question: {case['question']} context: {case.get('context', '')}"

            # Tokenize input
            inputs = tokenizer(input_text, return_tensors="pt", max_length=512, truncation=True).to(device)

            # Generate answer
            outputs = model.generate(
                **inputs,
                max_length=max_length,
                num_return_sequences=1,
                num_beams=4,
                early_stopping=True
            )

            # Decode answers
            predicted_answer = tokenizer.decode(outputs[0], skip_special_tokens=True)
            true_answer = case.get('answer', '')

            predicted_answers.append(predicted_answer)
            true_answers.append(true_answer)

    return predicted_answers, true_answers

def main():
    # Test cases with manual context and expected answers
    test_cases = [
        {
            'question': 'what is section 74',
            'context': '',
            'answer': 'Section 74 of the Indian Penal Code deals with the punishment of offences'
        },
        {
            'question': 'What is the extent of operation of the Indian Penal Code?',
            'context': 'The Indian Penal Code extends to the punishment of offences committed within India, and beyond but which by law may be tried within India. It also includes extension of the Code to extra-territorial offences.',
            'answer': 'The Indian Penal Code extends to offences committed within India and beyond, which can be tried within India'
        },
        {
            'question': 'What does the Indian Penal Code say about laws not to be affected by this Act?',
            'context': 'The Indian Penal Code specifies that certain laws are not to be affected by this Act.',
            'answer': 'The Act specifies that certain existing laws remain unaffected'
        },
        {
            'question': 'What does section 68 of the Indian Penal Code state about imprisonment?',
            'context': 'Section 68 states that imprisonment is to terminate on payment of fine.',
            'answer': 'Imprisonment terminates upon payment of fine'
        },
        {
            'question': 'How does the Indian Penal Code handle offences committed outside India?',
            'context': 'The Indian Penal Code extends to the punishment of offences committed within India, and beyond but which by law may be tried within India. It also includes extension of the Code to extra-territorial offences.',
            'answer': 'The Code extends to offences committed outside India that can be tried within India'
        }
    ]

    # Load the saved model
    model_path = "./t5_small_qa"  # Adjust path as needed
    tokenizer = T5Tokenizer.from_pretrained(model_path)
    model = T5ForConditionalGeneration.from_pretrained(model_path)

    # Evaluate model
    predicted_answers, true_answers = evaluate_model(model, tokenizer, test_cases)

    # Calculate accuracy
    accuracy_metrics = calculate_accuracy(true_answers, predicted_answers)

    # Print detailed results
    print("\n--- Model Accuracy Evaluation ---")
    print(f"Exact Match Accuracy: {accuracy_metrics['exact_match_accuracy']:.2f}%")
    print(f"Fuzzy Match Accuracy: {accuracy_metrics['fuzzy_match_accuracy']:.2f}%")
    print(f"Average ROUGE-L Score: {accuracy_metrics['average_rouge_l']:.2f}%")

    # Print detailed predictions
    print("\nDetailed Predictions:")
    for i, (true, pred) in enumerate(zip(true_answers, predicted_answers), 1):
        print(f"\n{i}. Question: {test_cases[i-1]['question']}")
        print(f"   True Answer   : {true}")
        print(f"   Predicted Answer: {pred}")

if __name__ == "__main__":
    main()

/usr/local/lib/python3.11/dist-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')



--- Model Accuracy Evaluation ---
Exact Match Accuracy: 0.00%
Fuzzy Match Accuracy: 20.00%
Average ROUGE-L Score: 37.94%

Detailed Predictions:

1. Question: what is section 74
   True Answer   : Section 74 of the Indian Penal Code deals with the punishment of offences
   Predicted Answer: 74

2. Question: What is the extent of operation of the Indian Penal Code?
   True Answer   : The Indian Penal Code extends to offences committed within India and beyond, which can be tried within India
   Predicted Answer: extension of the Code to extra-territorial offences

3. Question: What does the Indian Penal Code say about laws not to be affected by this Act?
   True Answer   : The Act specifies that certain existing laws remain unaffected
   Predicted Answer: certain laws are not to be affected by this Act.

4. Question: What does section 68 of the Indian Penal Code state about imprisonment?
   True Answer   : Imprisonment terminates upon payment of fine
   Predicted Answer: imprisonment is 